#### Dataset

The dataset chosen is the "Predict the positions and speeds of 600 satellites" dataset on Kaggle. This dataset predicts the position of a satellite at a given time 

The link to the dataset is as follows: https://www.kaggle.com/idawoodjee/predict-the-positions-and-speeds-of-600-satellites
 


In [14]:
## imports needed: 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,auc

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

In [8]:
# reading in the dataset and displaying the head of the dataset. 
dataset = pd.read_csv("jan_train.csv")
from dateutil import parser
from datetime import datetime

for e in range(len(dataset)): 
    dataset['epoch'][e] = datetime.strptime(dataset['epoch'][e], '%Y-%m-%d %H:%M:%S.%f')
    
dataset['epoch'] = dataset['epoch'] - dataset['epoch'][0]
for e in range(len(dataset)): 
    dataset['epoch'][e] = dataset['epoch'][e].total_seconds()



<ipython-input-8-495ce1ef4c0b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['epoch'][e] = datetime.strptime(dataset['epoch'][e], '%Y-%m-%d %H:%M:%S.%f')
<ipython-input-8-495ce1ef4c0b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['epoch'][e] = dataset['epoch'][e].total_seconds()
C:\Users\ajeet\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_wit

In [9]:
#display the tail of the dataset
dataset.tail()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
736,736,2.06021e+06,0,17323.752275,-2929.599994,40015.054282,-0.066983,2.651528,-0.240019,16677.210531,5234.016160,38340.904456,-0.330504,2.634282,-0.848806
737,737,2.06301e+06,0,16803.492692,4510.550397,38575.720092,-0.306878,2.639582,-0.792502,15395.263519,12449.405030,35146.024515,-0.586354,2.491844,-1.435062
738,738,2.06581e+06,0,15589.464016,11758.599852,35543.718040,-0.561601,2.510341,-1.375365,13380.486475,19062.449796,30278.225301,-0.852275,2.198419,-2.039662
739,739,2.06861e+06,0,13644.957424,18446.405418,30846.351215,-0.827062,2.233959,-1.978216,10617.516680,24593.429195,23717.669835,-1.117669,1.711798,-2.636755
740,740,2.07142e+06,0,10951.688066,24106.659914,24456.597220,-1.093516,1.769369,-2.577266,7135.816766,28430.515190,15552.366754,-1.359856,0.981096,-3.172296


#### Code to check for missing values and process them as needed
First, I will check for missing values: 

In [10]:
for var in dataset.columns:
    if any(dataset[var] == -1):
        print(var)
    else: 
        print("{} has no -1 values".format(var))
        



id has no -1 values
epoch has no -1 values
sat_id has no -1 values
x has no -1 values
y has no -1 values
z has no -1 values
Vx has no -1 values
Vy has no -1 values
Vz has no -1 values
x_sim has no -1 values
y_sim has no -1 values
z_sim has no -1 values
Vx_sim has no -1 values
Vy_sim has no -1 values
Vz_sim has no -1 values


There are no -1 values. 

### Neural Network Code 

Neural Network for the x position

In [15]:


X = dataset.loc[:, ['Vx', 'epoch']].values
y = dataset.x.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)
X_train.shape,X_test.shape,y_train.shape,y_test.shape
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

modelx = Sequential()

modelx.add(Dense(units=8,activation='relu'))
modelx.add(Dense(units=1))

modelx.compile(loss='mean_squared_error', optimizer='adam')

modelx.fit(x=X_train, 
          y=y_train, 
          epochs= 300,
          validation_data=(X_test, y_test), verbose=1
          )

Epoch 1/300
19/19 [==============================] - 1s 9ms/step - loss: 0.3603 - val_loss: 0.3143
Epoch 2/300
19/19 [==============================] - 0s 1ms/step - loss: 0.2919 - val_loss: 0.2532
Epoch 3/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2398 - val_loss: 0.2069
Epoch 4/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2012 - val_loss: 0.1744
Epoch 5/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1745 - val_loss: 0.1516
Epoch 6/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1576 - val_loss: 0.1378
Epoch 7/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1478 - val_loss: 0.1306
Epoch 8/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1430 - val_loss: 0.1271
Epoch 9/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1409 - val_loss: 0.1253
Epoch 10/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1398 - val_loss: 0.1244
Epoch 11/

Neural Network for the y position

In [16]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = dataset.loc[:, ['Vy', 'epoch']].values
y = dataset.y.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)
X_train.shape,X_test.shape,y_train.shape,y_test.shape
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

modely = Sequential()

modely.add(Dense(units=8,activation='relu'))
modely.add(Dense(units=1))

modely.compile(loss='mean_squared_error', optimizer='adam')

modely.fit(x=X_train, 
          y=y_train, 
          epochs= 300,
          validation_data=(X_test, y_test), verbose=1
          )

Epoch 1/300
19/19 [==============================] - 0s 7ms/step - loss: 1.0025 - val_loss: 0.8011
Epoch 2/300
19/19 [==============================] - 0s 2ms/step - loss: 0.7789 - val_loss: 0.6173
Epoch 3/300
19/19 [==============================] - 0s 2ms/step - loss: 0.6011 - val_loss: 0.4765
Epoch 4/300
19/19 [==============================] - 0s 3ms/step - loss: 0.4624 - val_loss: 0.3674
Epoch 5/300
19/19 [==============================] - 0s 2ms/step - loss: 0.3565 - val_loss: 0.2845
Epoch 6/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2766 - val_loss: 0.2240
Epoch 7/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2184 - val_loss: 0.1819
Epoch 8/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1783 - val_loss: 0.1552
Epoch 9/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1528 - val_loss: 0.1409
Epoch 10/300
19/19 [==============================] - 0s 2ms/step - loss: 0.1383 - val_loss: 0.1348
Epoch 11/

#### Code to display the coefficients

In [17]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = dataset.loc[:, ['Vz', 'epoch']].values
y = dataset.z.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2021)
X_train.shape,X_test.shape,y_train.shape,y_test.shape
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

modelz = Sequential()

modelz.add(Dense(units=8,activation='relu'))
modelz.add(Dense(units=1))

modelz.compile(loss='mean_squared_error', optimizer='adam')

modelz.fit(x=X_train, 
          y=y_train, 
          epochs= 300,
          validation_data=(X_test, y_test), verbose=1
          )

Epoch 1/300
19/19 [==============================] - 1s 6ms/step - loss: 0.6737 - val_loss: 0.5091
Epoch 2/300
19/19 [==============================] - 0s 1ms/step - loss: 0.4743 - val_loss: 0.3596
Epoch 3/300
19/19 [==============================] - 0s 2ms/step - loss: 0.3558 - val_loss: 0.2765
Epoch 4/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2900 - val_loss: 0.2347
Epoch 5/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2567 - val_loss: 0.2156
Epoch 6/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2415 - val_loss: 0.2058
Epoch 7/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2320 - val_loss: 0.2002
Epoch 8/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2262 - val_loss: 0.1961
Epoch 9/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2214 - val_loss: 0.1926
Epoch 10/300
19/19 [==============================] - 0s 2ms/step - loss: 0.2174 - val_loss: 0.1891
Epoch 11/

#### Code to predict to test set.

In [9]:
yhat = modelz.predict(np.array([[-0.2714602924326335, 0]]))
yhat

array([[0.1855419]], dtype=float32)